In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

# Always make it pretty.
plt.style.use('ggplot')

In [7]:
file = pd.read_csv('data/Collisions.csv')

In [8]:
file[file['SPEEDING'] == 'Y'].shape[0]
#Number of Accidents with speeding involved

9492

In [9]:
file[(file['SPEEDING'] == 'Y') & (file['FATALITIES'] > 0)].shape[0]

85

In [10]:
file[file['FATALITIES'] > 0].shape[0]

328

In [11]:
print(file['INCDTTM'].isnull().sum())
file['UNDERINFL'].isnull().sum()


0


25721

In [12]:
file[(file['UNDERINFL'] == 'Y') | (file['UNDERINFL'] == '1' )].shape[0]

9315

In [13]:
file[file['SERIOUSINJURIES'] > 0].shape[0]
file['UNDERINFL'].unique()
drunkdracc = file[((file['UNDERINFL'] == 'Y') | (file['UNDERINFL'] == '1'))].shape[0]

In [14]:
infl = file[((file['UNDERINFL'] == 'Y') & (file['FATALITIES'] > 0) )| ((file['UNDERINFL'] == '1') & (file['FATALITIES'] > 0))].shape[0]


underinflratio = infl / file[file['FATALITIES'] > 0 ].shape[0]

noninfl = file[((file['UNDERINFL'] == 'N') & (file['FATALITIES'] > 0) )| ((file['UNDERINFL'] == '0') & (file['FATALITIES'] > 0))].shape[0]

noninfluratio = noninfl / file[file['FATALITIES'] > 0].shape[0]
#print(noninfluratio)
#print(underinflratio)
#print(file[(file['FATALITIES'] > 0) & (file['ADDRTYPE'] == 'Block')].shape[0])
#print(file[(file['FATALITIES'] > 0) & (file['ADDRTYPE'] == 'Intersection')].shape[0])
#print(file[(file['FATALITIES'] > 0) & (file['LIGHTCOND'] == 'Daylight')].shape[0])
print('drunk driving deaths / drunk driving accidents: ',(infl/drunkdracc))
nondrunkdrivingacccsdeath = file[((file['UNDERINFL'] == 'N') & (file['FATALITIES'] > 0) )| ((file['UNDERINFL'] == '0') & (file['FATALITIES'] > 0))].shape[0]
nondrunkdrivingacc = file[(file['UNDERINFL'] == 'N') | (file['UNDERINFL'] == '0')].shape[0]
print(nondrunkdrivingacc)
print('non drunk driving deaths / non drunk driving accidents: ',(nondrunkdrivingacccsdeath/nondrunkdrivingacc))
denominator = drunkdracc * nondrunkdrivingacc 
numerator = drunkdracc + nondrunkdrivingacc
shared_sample_freq = (nondrunkdrivingacccsdeath + infl) / denominator
shared_sample_variance = numerator * (shared_sample_freq * (1 - shared_sample_freq)) / denominator
difference_in_proportions = stats.norm(0, np.sqrt(shared_sample_variance))

drunk driving deaths / drunk driving accidents:  0.00966183574879227
177568
non drunk driving deaths / non drunk driving accidents:  0.001267120201838169


In [127]:
seriousinjury = file[((file['UNDERINFL'] == 'Y') & (file['SERIOUSINJURIES'] > 0) )| ((file['UNDERINFL'] == '1') & (file['SERIOUSINJURIES'] > 0))].shape[0]
totalseriousinjury = file[file['SERIOUSINJURIES'] > 0].shape[0]
seriousinjurynondrunk = file[((file['UNDERINFL'] == 'N') & (file['SERIOUSINJURIES'] > 0) )| ((file['UNDERINFL'] == '0') & (file['SERIOUSINJURIES'] > 0))].shape[0]
print(seriousinjury/totalseriousinjury)
print('serious injury driving accidents invoving drunk / drunk driving accidents: ',seriousinjury/drunkdracc)
print('serious injury driving accidents not involving drunk/ non drunk driving accidents: ',seriousinjurynondrunk/nondrunkdrivingacc)

0.14145516074450085
serious injury driving accidents invoving drunk / drunk driving accidents:  0.04487385936661299
serious injury driving accidents not involving drunk/ non drunk driving accidents:  0.014276220940710039


In [126]:
injury = file[((file['UNDERINFL'] == 'Y') & (file['INJURIES'] > 0) )| ((file['UNDERINFL'] == '1') & (file['INJURIES'] > 0))].shape[0]
injurynondrunk = file[((file['UNDERINFL'] == 'N') & (file['INJURIES'] > 0) )| ((file['UNDERINFL'] == '0') & (file['INJURIES'] > 0))].shape[0]
totalinjury = file[file['INJURIES'] > 0].shape[0]
print(injury/totalinjury)
print(injury/drunkdracc)
print(injurynondrunk/nondrunkdrivingacc)

0.06567435901758172
0.41825013419216317
0.3051563344746801


In [138]:
print('car accidents involving drunk driver that injury someone',injury)
print('car accidents involving non drunk driver that injury someone',injurynondrunk)
print('car accidents involving drunk driver',drunkdracc)
print('car accidents involving drunk driver',nondrunkdrivingacc)
denominator = drunkdracc * nondrunkdrivingacc 
numerator = drunkdracc + nondrunkdrivingacc

car accidents involving drunk driver that injury someone 3896
car accidents involving non drunk driver that injury someone 54186
car accidents involving drunk driver 9315
car accidents involving drunk driver 177568


In [24]:
file[(file['SPEEDING'] == 'Y') & (file['SERIOUSINJURIES'] > 0)].shape[0]

377

In [25]:
file[file['INJURIES'] > 0].shape[0]

59323

In [26]:
file[(file['SPEEDING'] == 'Y') & (file['INJURIES'] > 0)].shape[0]

3821

In [66]:
file.groupby('ST_COLCODE').count()

,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,...,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
ST_COLCODE,,,,,,,,,,,,,,,,,,,,,
,14970,14970,16329,16329,16329,16329,16329,15098,3385,15047,...,3,3,3,0,888,0,0,16329,16329,16329
0,3308,3308,3349,3349,3349,3349,3349,3346,1721,3324,...,3342,3345,3342,755,1975,47,3349,3349,3349,3349
1,1217,1217,1223,1223,1223,1223,1223,1222,1078,1218,...,1222,1223,1222,795,729,8,1223,1223,1223,1223
10,33595,33595,33804,33804,33804,33804,33804,33739,28221,33686,...,33789,33799,33771,13,20250,1106,33804,33804,33804,33804
11,11857,11857,12327,12327,12327,12327,12327,12223,1739,12223,...,12322,12324,12314,4,7236,262,12327,12327,12327,12327
12,2288,2288,2335,2335,2335,2335,2335,2316,225,2311,...,2330,2332,2328,0,1483,57,2335,2335,2335,2335
13,7011,7011,7412,7412,7412,7412,7412,7343,806,7341,...,7405,7409,7405,1,4087,587,7412,7412,7412,7412
14,24492,24492,25203,25203,25203,25203,25203,25029,2922,25020,...,25192,25190,25181,1,16042,1374,25203,25203,25203,25203
15,2998,2998,3017,3017,3017,3017,3017,3006,2209,3006,...,3017,3015,3014,1,1900,48,3017,3017,3017,3017
